In [1]:
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [2]:
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
print("[INFO] loading images...")
img_path = 'Images-processed'
img_COVID_paths = list(paths.list_images(os.path.join(img_path,'CT_COVID')))
img_NonCOVID_paths = list(paths.list_images(os.path.join(img_path,'CT_NonCOVID')))

[INFO] loading images...


In [4]:
img_COVID_paths[0]

'Images-processed\\CT_COVID\\2019-novel-Coronavirus-severe-adult-respiratory-dist_2020_International-Jour-p3-89%0.png'

In [3]:
data = []
labels = []
# deal with the img_COVID
for imagePath in img_COVID_paths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath) ## all the images are stored as a numpy array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# deal with the img_NonCOVID
for imagePath in img_NonCOVID_paths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath) ## all the images are stored as a numpy array
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.array(cv2.resize(image, (224, 224)))
    #image_data = image.reshape(image.shape[0]*image.shape[1], )
    #image_data = image.flatten()
    #image_data = image.reshape(-1)
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)

[INFO] loading images...


In [4]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer() 
labels_num = lb.fit_transform(labels).reshape(-1)
# re_label = lb.inverse_transform(Y) # 还原label
labels_num

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
data_list = []
for image in data:
    #image_data = image.reshape(image.shape[0]*image.shape[1], )
    image_data = image.flatten()
    data_list.append(image_data)

In [6]:
data_list[0].shape

(50176,)

In [7]:
data_np = np.array(data_list)

In [8]:
data_np[0].shape

(50176,)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_np, labels_num, test_size=0.2)

In [11]:
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

In [10]:
from sklearn import svm
svm_clf = svm.SVC().fit(X_train, y_train)

In [13]:
svm_acc = accuracy_score(y_test,svm_clf.predict(X_test))
svm_auc = roc_auc_score(y_test,svm_clf.predict(X_test))
svm_precision = precision_score(y_test,svm_clf.predict(X_test))
svm_recall = recall_score(y_test,svm_clf.predict(X_test))
svm_f1 = recall_score(y_test,svm_clf.predict(X_test))
print('SVM :recall: {:.4f}, precision: {:.4f},F1: {:.4f}, accuracy: {:.4f}, AUC: {:.4f}'.format(
        svm_recall, svm_precision, svm_f1, svm_acc, svm_auc))

recall: 0.8125, precision: 0.7065,F1: 0.8125, accuracy: 0.7200, AUC: 0.7134


In [14]:
from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression().fit(X_train, y_train)

In [15]:
lg_acc = accuracy_score(y_test,lg_clf.predict(X_test))
lg_auc = roc_auc_score(y_test,lg_clf.predict(X_test))
lg_precision = precision_score(y_test,lg_clf.predict(X_test))
lg_recall = recall_score(y_test,lg_clf.predict(X_test))
lg_f1 = recall_score(y_test,lg_clf.predict(X_test))
print('LogisticRegression :recall: {:.4f}, precision: {:.4f},F1: {:.4f}, accuracy: {:.4f}, AUC: {:.4f}'.format(
        lg_recall, lg_precision, lg_f1, lg_acc, lg_auc))

LogisticRegression :recall: 0.6625, precision: 0.6463,F1: 0.6625, accuracy: 0.6267, AUC: 0.6241


In [20]:
from sklearn.tree import DecisionTreeClassifier  
tree_clf = DecisionTreeClassifier().fit(X_train, y_train)

In [21]:
tree_acc = accuracy_score(y_test,tree_clf.predict(X_test))
tree_auc = roc_auc_score(y_test,tree_clf.predict(X_test))
tree_precision = precision_score(y_test,tree_clf.predict(X_test))
tree_recall = recall_score(y_test,tree_clf.predict(X_test))
tree_f1 = recall_score(y_test,tree_clf.predict(X_test))
print('DecisionTreeClassifier :recall: {:.4f}, precision: {:.4f},F1: {:.4f}, accuracy: {:.4f}, AUC: {:.4f}'.format(
        tree_recall, tree_precision, tree_f1, tree_acc, tree_auc))

DecisionTreeClassifier :recall: 0.6500, precision: 0.6933,F1: 0.6500, accuracy: 0.6600, AUC: 0.6607


In [22]:
from sklearn.ensemble import RandomForestClassifier 
rftree_clf = RandomForestClassifier().fit(X_train, y_train)

In [23]:
rftree_acc = accuracy_score(y_test,rftree_clf.predict(X_test))
rftree_auc = roc_auc_score(y_test,rftree_clf.predict(X_test))
rftree_precision = precision_score(y_test,rftree_clf.predict(X_test))
rftree_recall = recall_score(y_test,rftree_clf.predict(X_test))
rftree_f1 = recall_score(y_test,rftree_clf.predict(X_test))
print('RandomForestClassifier :recall: {:.4f}, precision: {:.4f},F1: {:.4f}, accuracy: {:.4f}, AUC: {:.4f}'.format(
        rftree_recall, rftree_precision, rftree_f1, rftree_acc, rftree_auc))

RandomForestClassifier :recall: 0.8500, precision: 0.7640,F1: 0.8500, accuracy: 0.7800, AUC: 0.7750
